In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from vadereanalysistool import ScenarioOutput, VadereProject
from scipy.optimize import curve_fit

sns.set(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.0})

def plotEvolution(vproject, ending, yval, ylabel):
    plots = []
    i = 1
    rows = len(vproject.output_dirs);
    plt.figure(1, figsize=(30, 10*rows))
    for outStr in project.output_dirs :
        out = project.output_dirs[outStr]
        dataFrames = []
        for fileStr in out.files :
            if fileStr.endswith(ending) :
                ndf = pd.DataFrame();
                df = out.files[fileStr]()
                #df = df[df.velocity > 0]
                ndf['density'] = df.density.astype(float)
                ndf['velocity'] = df.velocity.astype(float)
                ndf['timeStep'] = df.timeStep.astype(int)
                ndf['scenario'] = [out.scenario['name']] * len(df.density)
                dataFrames.append(ndf)
            #concatFrames = pd.concat(dataFrames)
        plt.subplot(rows, 1, i)
        plt.title(out.scenario['name'])
        plt.xlabel('timeStep')
        plt.ylabel(ylabel)
        sns.lineplot(x='timeStep', y=yval, data=pd.concat(dataFrames))
        i = i + 1

def plotFundamentalDiagram(vproject, ending):
    dataFrames = []
    for outStr in project.output_dirs :
        out = project.output_dirs[outStr]
        for fileStr in out.files :
            if fileStr.endswith(ending) :
                ndf = pd.DataFrame();
                df = out.files[fileStr]()
                ndf['density'] = df.density.astype(float)
                ndf['velocity'] = df.velocity.astype(float)
                ndf['timeStep'] = df.timeStep.astype(int)
                ndf['scenario'] = [fileStr] * len(df.velocity)
                #ndf = ndf[ndf.timeStep > 100]
                #ndf = ndf[ndf.timeStep < 500]
                ndf = ndf[ndf.density < 4]
                ndf = ndf[ndf.velocity > 0]
                dataFrames.append(ndf)
    concatFrames = pd.concat(dataFrames)
    g = sns.relplot(x="density", y="velocity", hue="scenario", data=concatFrames,
                height=10, aspect=1)
    g.set(xticks=[0,0.5,1,1.5,2,2.5,3,3.5,4], yticks=[0,0.5,1,1.5,2,2.5]);
    g.savefig("./"+vproject.project_name+"_"+ending+"_fd"+".pdf", bbox_inches='tight')
    #axis = g.axes
    #axes.set_xticks([0,1,2,3,4,5,6])
    #axes.set_yticks([0,0.5,1,1.5,2,2.5])
    #axes.set_xlim(0,6)
    #axes.set_ylim(0,2.5)
    
def plotFundamentalDiagramScatter(vproject, ending, sep=False, width = 10, height = 5):
    dataFrames = []
    index = 0
    cols = 3
    rows = len(vproject.output_dirs) / cols + 1
    if not sep :
        fig, axs = plt.subplots(int(rows), int(cols), figsize=(height*rows, width*cols), sharex=False, sharey=True)       
    for outStr in vproject.output_dirs :
        out = vproject.output_dirs[outStr]
        for fileStr in out.files :
            if fileStr.endswith(ending) :
                if not sep : 
                    axes = axs[int(index / cols), int(index % cols)]
                else :
                    fig = plt.figure(1, figsize=(width, height))
                    axes = plt.axes()
                    fig.add_axes(axes)
                ndf = pd.DataFrame();
                df = out.files[fileStr]()
                ndf['density'] = df.density.astype(float)
                ndf['velocity'] = df.velocity.astype(float)
                ndf['timeStep'] = df.timeStep.astype(int)
                ndf['scenario'] = [out.scenario['name']] * len(df.velocity)
                ndf = ndf[ndf.density < 4]
                #ndf = ndf[ndf.timeStep > 100]
                #ndf = ndf[ndf.timeStep < 300]
                #plt.scatter()
                #axes.set_title(out.scenario['name'])
                axes.set_xlabel('density')
                axes.set_ylabel('velocity')
                axes.set_xticks([0,1,2,3,4,5,6])
                axes.set_yticks([0,0.5,1,1.5,2,2.5])
                axes.set_xlim(0,6)
                axes.set_ylim(0,2.5)
                axes.scatter(ndf['density'], ndf['velocity'], s=0.7, marker='*', color='#555555')
                wm = plotWeidmann(axes)
                popt, pcov = curve_fit(kladek, ndf['density'], ndf['velocity'], p0=(1.34, 1.913, 5.4))
                print(str(popt[0]) + "," + str(popt[1]) + "," + str(popt[2]))
                xx = np.linspace(0.1, 6, 1000)
                yy = kladek(xx, *popt)
                axes.plot(xx, yy, '--', c=sns.color_palette().as_hex()[1])     
                axes.legend(['Weidmann', 'regression', 'Simulated data'])
                index = index + 1;
                if sep :
                    fig.savefig("./"+out.scenario['name']+"_"+ending+"_fd"+".pdf", bbox_inches='tight')
                    plt.show()
    if not sep :
        fig.savefig("./"+vproject.project_name+"_"+ending+"_fd"+".pdf", bbox_inches='tight')
        

# Load the Vadere project

In [ ]:
projectFolder = "./../../../../Scenarios/ModelCalibration/TestOSM_zhang-2011/"
project = VadereProject(projectFolder)
#out = project.named_output.C_050_180_180_2018_11_26_16_30_29_355() 

## Method A Plots

In [ ]:
# transform data frame
plotFundamentalDiagram(project, "aTimeStep.fundamentalDiagram")

#### Method B Plots

In [ ]:
plotFundamentalDiagram(project, "bPedestrian.fundamentalDiagram")

## Method C Plots (classical density)

In [ ]:
plotFundamentalDiagram(project, "cTimeStep.fundamentalDiagram")

## Method E Plots (voronoi density)

In [ ]:
plotFundamentalDiagram(project, "eTimeStep.fundamentalDiagram")

## Method D Plots (voronoi density)

In [ ]:
plotFundamentalDiagram(project, "dTimeStep.fundamentalDiagram")

In [ ]:
plotEvolution(project, "aTimeStep.fundamentalDiagram", 'density', 'density')

In [ ]:
import matplotlib.pyplot as plt

coord1 = [[23.926582253991363,11.311616564587455],[24.526824188631515,11.346725287186784],[24.52439283212948,11.89],[23.80719398422859,11.89]]
coord1.append(coord1[0])
coord2 = [[22.0,10.1], [24.0,10.1], [24.0,11.9], [22.0,11.9], [22.0,10.1]]
coord2.append(coord2[0])
cap = [[23.926582253991363,11.311616564587455],[24.000000000000014,11.315910838495633],[24.000000000000025,11.889999999999956],[23.80719398422859,11.89]]
cap.append(cap[0])

xs1, ys1 = zip(*coord1) #create lists of x and y values
xs2, ys2 = zip(*coord2) 
xs3, ys3 = zip(*cap) 
plt.figure(figsize=(10,10))
plt.plot(xs1,ys1)
plt.plot(xs2,ys2)
plt.plot(xs3,ys3)
plt.show() # if you need...


In [ ]:
import matplotlib.pyplot as plt

coord = [[1,1], [2,1], [2,2], [1,2], [0.5,1.5]]
coord.append(coord[0]) #repeat the first point to create a 'closed loop'

xs, ys = zip(*coord) #create lists of x and y values

plt.figure()
plt.plot(xs,ys) 
plt.show() # if you need..